In [2]:
# import numpy as np
# import re
# class Tokenizer:
    
#     def __init__(self, vocab):
#         self.str_to_int = vocab # vocab is a mapping of tokens to token IDs
#         self.int_to_str = {i:s for i,s in vocab.items()}
    
#     def encode(self, text): # convert test to token Ids
#         processed = re.split(r'([:,.?_=+()-!";\'@#$%&*]|--|\s)', text)
#         processed = [item.strip() for item in processed if item.strip()] # removing the blank spaces
#         processed = [
#             item if item in self.str_to_int
#             else "<unk>" for item in preprocessed
#         ]
#         ids = [self.str__to_int[s] for s in processed]
#         return ids
    
#     def decode(self, ids): # convert token IDs to text
#         text = " ".join([self.int_to_str[i] for i in ids])
#         text = re.sub(r'\s+([,.?();":!])', r'\1',text)
#         return text 
    

In [3]:
!pip install tiktoken

In [4]:
# byte pair tokenization
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [5]:
import requests

url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"
response = requests.get(url)
if response.status_code == 200:
    with open("the-verdict.txt", "w", encoding="utf-8") as f:
        f.write(response.text)
    print("File downloaded and saved!")
else:
    print("Failed to download file:", response.status_code)
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    text = f.read()
print(text[:200])


File downloaded and saved!
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a


In [6]:
import torch
print(torch.__file__)        # Path to actual torch package
print(torch.__version__)     # Should show version string
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0)) 

C:\Users\Kabir Ghuge\anaconda3\envs\torch-gpu\lib\site-packages\torch\__init__.py
2.5.1
True
NVIDIA GeForce RTX 4060 Laptop GPU


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
class Dataprep(Dataset):
    
    def __init__(self, text,tokenizer, max_length, stride):
        self.input_tensor = []
        self.target_tensor = []
        
        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
        
        for i in range(0, len(token_ids) - max_length - stride + 1, stride):
            input_window = token_ids[i:i + max_length]
            target_window = token_ids[i + stride:i + max_length + stride]

            if len(target_window) == max_length:
                self.input_tensor.append(torch.tensor(input_window))
                self.target_tensor.append(torch.tensor(target_window))

            
    def __len__(self):
        return len(self.input_tensor)
    
    def __getitem__(self, idx):
        return self.input_tensor[idx] , self.target_tensor[idx]

In [8]:
def create_dataloader(text, batch_size=4, max_length=256, stride=128, drop_last=True, shuffle=False):
    
        tokenizer = tiktoken.get_encoding("gpt2")
        dataset = Dataprep(text, tokenizer, max_length, stride)
        
        dataloader = DataLoader(   
            dataset,
            batch_size=batch_size,
            drop_last=drop_last,
            shuffle=shuffle
            )
        return dataloader

In [9]:
print(len(tokenizer.encode(text)))
print(len(text))

5145
20479


In [10]:
# # creating the inputs and targets tensor of dims 8x4
# dataloader = create_dataloader(text, batch_size=8, max_length=4, stride=4, shuffle=False)
# data_iter = iter(dataloader)
# inputs , targets = next(data_iter)

In [11]:
# # vector embeddings
# vocab_size = 50257 # gpt2 had 50257 tokens
# embed_dim = 256
# vec_embed_layer = torch.nn.Embedding(vocab_size, embed_dim)
# vector_embeddings = vec_embed_layer(inputs)

# # positional encodings
# max_length = 4
# pos_encoding_layer = torch.nn.Embedding(max_length, embed_dim)
# pos_encodings = pos_encoding_layer(torch.arange(max_length))

# # input embeddings 
# input_embed = vector_embeddings + pos_encodings
# input_embed.shape

In [12]:
# class self_attention(torch.nn.Module):
    
#     def __init__(self, din, dout, qkv_bias=False):
#         super().init()
#         self.Wq = torch.nn.Linear(din, dout, bias=qkv_bias)
#         self.Wk = torch.nn.Linear(din, dout, bias=qkv_bias)
#         self.Wv = torch.nn.Linear(din, dout, bias=qkv_bias)

        
#     def forward(self, x):
#         Q = self.Wq(x)
#         K = self.Wk(x)
#         V = self.Wv(x)
        
#         attention_scores = Q @ K.T
#         atten_weights = torch.softmax(attention_scores / K.shape[-1]**0.5, dim = -1)
#         context_vector = atten_weights @ V
#         return context_vector

In [13]:
# class CausalAttention(torch.nn.Module):
    
#     def __init__(self, din, dout, context_length, dropout_rate, qkv_bias=False):
#         super.init()
#         self.Wq = torch.nn.Linear(din, dout, bias=qkv_bias) # dout is the user defiend but din should strictly be equal to dim = embed_size for matrix mul
#         self.Wk = torch.nn.Linear(din, dout, bias=qkv_bias)
#         self.Wv = torch.nn.Linear(din, dout, bias=qkv_bias)
#         self.dropout = torch.nn.Dropout(dropout_rate)
#         self.register_buffer(torch.triu(torch.ones('mask',context_length, context_length), diagonal=1))
        
#     def forward(self, x):
#         batch, no_tokens, din = x.shape
#         Q = self.Wq(x)
#         K = self.Wk(x)
#         V = self.Wv(x)
        
#         attention_scores = Q @ K.transpose(1,2)
#         masked = attention_scores.masked_fill_(mask.bool()[:no_tokens, no_tokens], -torch.inf)
#         atten_weights = torch.softmax(masked / K.shape[-1] ** 0.5, dim=-1)
#         atten_weights = self.dropout(atten_weights)
#         context_vector = atten_weights @ V
        
#         return context_vector

In [14]:
# class MultiHeadAttention(torch.nn.Module):
    
#     def __init__(self, din, dout, context_length, num_heads, dropout_rate):
#         super().__init__()
#         self.head = torch.Modulelist(
#             [CausalAttention(din, dout, context_length, dropout_rate, qkv_bias) for _ in range(num_heads)]
#         )
        
#     def forward(self, x):
#         return torch.cat( [head[x] for head in self.head] , dim=-1)

In [15]:
import torch.nn as nn
class MaskedMultiheadAttention(nn.Module):
    
    def __init__(self, din, dout, context_length, num_heads, dropout_rate, qkv_bias=False):
        super().__init__()
        self.num_heads = num_heads
        self.dout = dout
        self.head_dim  = dout // num_heads
        self.Wq = torch.nn.Linear(din, dout, bias=qkv_bias) # dout is the user defiend but din should strictly be equal to dim = embed_size for matrix mul
        self.Wk = torch.nn.Linear(din, dout, bias=qkv_bias)
        self.Wv = torch.nn.Linear(din, dout, bias=qkv_bias)
        self.out_proj = torch.nn.Linear(dout, dout)
        self.dropout = torch.nn.Dropout(dropout_rate)
        
        mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
        self.register_buffer("mask", mask)
        
    def forward(self, x):
        batch, n_tokens, din = x.shape
        Q = self.Wq(x)
        K = self.Wk(x)
        V = self.Wv(x)
        
        Q = Q.view(batch, n_tokens, self.num_heads, self.head_dim)
        V = V.view(batch, n_tokens, self.num_heads, self.head_dim)
        K = K.view(batch, n_tokens, self.num_heads, self.head_dim)
        
        Q = Q.transpose(1,2)
        V = V.transpose(1,2)
        K = K.transpose(1,2)
        
        attention_scores = Q @ K.transpose(2,3)
        masked = attention_scores.masked_fill_(self.mask.bool()[:n_tokens, :n_tokens], -torch.inf)
        atten_weights = torch.softmax(masked/self.head_dim ** 0.5, dim=-1)
        atten_weights = self.dropout(atten_weights)
        context_vector = (atten_weights @ V).transpose(1,2)
        context_vector = context_vector.contiguous().view(batch, n_tokens, self.dout)
        context_vector =  self.out_proj(context_vector)
        
        return context_vector

In [16]:
cfg = {
    "vocab_size":50257,
    "context_length":256,
    "emb_dim":768,
    "n_layer":12,
    "n_head":12,
    "dropout":0.1,
    "qkv_bias":False
}

In [17]:
import torch.nn as nn
import numpy as np
class LayerNorm(nn.Module):
    
    def __init__(self, embed_dim):
        super().__init__()
        self.scale = nn.Parameter(torch.ones(embed_dim))
        self.shift = nn.Parameter(torch.zeros(embed_dim))
        self.espilon = 1e-7
        
    def forward(self, x):
        mean = x.mean(axis=-1, keepdims=True)
        var = x.var(axis=-1, keepdims=True, unbiased=True)
        norm_x = (x - mean)/torch.sqrt(var + self.espilon )
        return self.scale * norm_x + self.shift
    
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi)) * (x + 0.044715 * x ** 3)))

    
class FeedForward(nn.Module):
    
    def __init__(self, cfg):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(cfg['emb_dim'], 4*cfg['emb_dim']),
            GELU(),
            nn.Linear(4*cfg['emb_dim'], cfg['emb_dim'])
        )
        
    def forward(self, x):
        return self.layer(x)

In [18]:
class Transformer(nn.Module):
    
    def __init__(self, cfg):
        super().__init__()
        self.attention = MaskedMultiheadAttention(
            din = cfg["emb_dim"], 
            dout = cfg["emb_dim"], 
            context_length = cfg["context_length"], 
            num_heads = cfg["n_head"], 
            dropout_rate = cfg["dropout"],
            qkv_bias = cfg["qkv_bias"]
        )
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(embed_dim = cfg["emb_dim"])
        self.norm2 = LayerNorm(embed_dim = cfg["emb_dim"])      
        self.drop = nn.Dropout(cfg["dropout"])
        
    def forward(self, x):
        shortcut = x
        x = self.norm1(x) # pre normalization results in better reults than post normalization
        x = self.attention(x)
        x = self.drop(x)
        x = x + shortcut
        
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop(x)
        x = x + shortcut
        
        return x

In [19]:
class GPT(nn.Module):
    
    def __init__(self, cfg):
        super().__init__()
        self.embed_layer = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_layer = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.dropout = nn.Dropout(cfg["dropout"])
        
        self.tf_blocks = nn.Sequential(*[Transformer(cfg) for _ in range(cfg["n_layer"])])
        
        self.norm = LayerNorm(cfg["emb_dim"])
        
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)
        
    def forward(self, in_ids):
        batch, seq_len = in_ids.shape
        token_embed = self.embed_layer(in_ids)
        pos_ids = torch.arange(seq_len, device=in_ids.device)
        pos_encoding = self.pos_layer(pos_ids)
        x = token_embed + pos_encoding
        x = self.dropout(x)
        x = self.tf_blocks(x)
        x = self.norm(x)
        logits = self.out_head(x)
        return logits     

torch.manual_seed(123)
model = GPT(cfg)
model.eval();  # Disable dropout during inference

In [20]:
# def generate_next_tokens(idx, model, max_new_tokens, context_length, temperature=1.0, top_k=50, top_p=0.9):
#     idx = idx.to(dtype=torch.long, device=model.pos_layer.weight.device)

#     for _ in range(max_new_tokens):
#         idx_cond = idx[:, -context_length:]
#         with torch.no_grad():
#             logits = model(idx_cond)

#         logits = logits[:, -1, :] / temperature
#         probs = torch.softmax(logits, dim=-1)

#         # Top-k filtering
#         if top_k is not None:
#             top_k = min(top_k, probs.size(-1))  # Safety
#             top_k_vals, top_k_indices = torch.topk(probs, top_k)
#             probs_filtered = torch.zeros_like(probs).scatter_(1, top_k_indices, top_k_vals)
#             probs = probs_filtered / probs_filtered.sum(dim=-1, keepdim=True)

#         # Nucleus (top-p) filtering
#         if top_p is not None:
#             sorted_probs, sorted_indices = torch.sort(probs, descending=True)
#             cum_probs = sorted_probs.cumsum(dim=-1)

#             # Create mask for tokens to keep
#             keep_mask = cum_probs <= top_p
#             # Always keep at least 1 token
#             keep_mask[..., 0] = 1

#             filtered_probs = torch.zeros_like(probs)
#             filtered_probs.scatter_(1, sorted_indices, keep_mask.float() * sorted_probs)
#             probs = filtered_probs / filtered_probs.sum(dim=-1, keepdim=True)

#         next_token = torch.multinomial(probs, num_samples=1)
#         idx = torch.cat((idx, next_token), dim=1)

#     return idx
def generate_next_tokens(idx, model, max_new_tokens, context_length, temperature=1.0):
    idx = idx.to(dtype=torch.long, device=model.pos_layer.weight.device)
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_length:]
        with torch.no_grad():
            logits = model(idx_cond)
        last_token_logits = logits[:, -1, :]
        probs = torch.softmax(last_token_logits / temperature, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)  # sampling
        idx = torch.cat((idx, next_token), dim=1)
    return idx


In [21]:
model_params = sum(p.numel() for p in model.parameters())
print(f"total model parameters: {model_params:,}")

total model parameters: 162,419,712


In [22]:
def crossentropyloss(input_batch, target_batch, model, device):
    input_batch , target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = nn.functional.cross_entropy(logits.flatten(0,1), target_batch.flatten())
    return loss

def calc_loss(dataloader, device, model, num_batches=None):
    total_loss = 0
    if len(dataloader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(dataloader)
    else :
        num_batches = min(num_batches, len(dataloader))
    for i, (input_batch, target_batch) in enumerate(dataloader):
        if i < num_batches:
            loss = crossentropyloss(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else :
            break
    return  total_loss / num_batches

In [23]:
train_split = 0.9
data_split  = int(train_split * len(text))
train_data = text[:data_split]
val_data = text[data_split:]

torch.manual_seed(123)
train_loader = create_dataloader(train_data, batch_size=2, max_length=cfg["context_length"], stride=cfg["context_length"], drop_last=True, shuffle=True)
val_loader = create_dataloader(val_data, batch_size=2, max_length=cfg["context_length"], stride=cfg["context_length"], drop_last=False, shuffle=False)

for x,y in train_loader:
    print(f"train_loader:{x.shape, y.shape}")
    
for x,y in val_loader:
    print(f"val_loader:{x.shape, y.shape}")


train_loader:(torch.Size([2, 256]), torch.Size([2, 256]))
train_loader:(torch.Size([2, 256]), torch.Size([2, 256]))
train_loader:(torch.Size([2, 256]), torch.Size([2, 256]))
train_loader:(torch.Size([2, 256]), torch.Size([2, 256]))
train_loader:(torch.Size([2, 256]), torch.Size([2, 256]))
train_loader:(torch.Size([2, 256]), torch.Size([2, 256]))
train_loader:(torch.Size([2, 256]), torch.Size([2, 256]))
train_loader:(torch.Size([2, 256]), torch.Size([2, 256]))
val_loader:(torch.Size([1, 256]), torch.Size([1, 256]))


In [24]:
train_tokens = 0
for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()

val_tokens = 0
for input_batch, target_batch in val_loader:
    val_tokens += input_batch.numel()

print("Training tokens:", train_tokens)
print("Validation tokens:", val_tokens)
print("All tokens:", train_tokens + val_tokens)

Training tokens: 4096
Validation tokens: 256
All tokens: 4352


In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes


torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss(train_loader, device, model)
    val_loss = calc_loss(val_loader, device,  model)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Training loss: 10.987075567245483
Validation loss: 11.03717041015625


In [26]:
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

In [27]:
def evaluate_model(model, device, eval_iter, train_loader, val_loader):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss(train_loader, device, model, num_batches=eval_iter)
        val_loss = calc_loss(val_loader, device, model, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

In [28]:
def generate_print_sample(model, device, tokenizer, start_context):
    model.eval()
    context_size = model.pos_layer.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_next_tokens(
    # model=model, 
    # idx=encoded, 
    # max_new_tokens=50, 
    # context_length=context_size, 
    # temperature=0.95, 
    # top_k=45, 
    # top_p=0.9
            idx=encoded, model=model, max_new_tokens=50, context_length=context_size
)
    decoded = token_ids_to_text(token_ids, tokenizer)
    print(decoded.replace("\n", " "))
    model.train()

In [29]:
def train_model(model, train_loader, val_loader, device, eval_iter, eval_freq, num_epochs, start_context, optimizer,tokenizer):
    
    train_loss, val_loss, tokens_seen = [], [], []
    tokens_seen1, global_step = 0, -1
    
    for epoch in range(num_epochs):
        model.train()
        
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad()
            loss = crossentropyloss(input_batch, target_batch, model, device)
            loss.backward()
            optimizer.step()
            tokens_seen1 += input_batch.numel()
            global_step += 1
            
            if global_step % eval_freq == 0 :
                train_loss1, val_loss1 = evaluate_model(model, device, eval_iter, train_loader, val_loader)
                train_loss.append(train_loss1)
                val_loss.append(val_loss1)
                tokens_seen.append(tokens_seen1)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                    f"Train loss {train_loss[-1]:.3f}, Val loss {val_loss[-1]:.3f}")

                
        generate_print_sample(model, device, tokenizer, start_context)
    return train_loss, val_loss, tokens_seen

In [30]:

import time
start_time = time.time()

torch.manual_seed(123)
model = GPT(cfg)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_loss, val_loss, tokens_seen = train_model(
    model, train_loader, val_loader, device, eval_iter=5, eval_freq=5,
    num_epochs=num_epochs, start_context="Every effort moves you", optimizer=optimizer, tokenizer=tokenizer
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 9.912, Val loss 10.183
Ep 1 (Step 000005): Train loss 8.039, Val loss 8.349
Every effort moves you ofgurets  spaciousatsu226 gh. Beansverted Stefan kickoffAb Peg the thehh superbNet,Plex--gottenciation YEAR ThousandsixtタBle brilliantlyitiz crashing .except.lopp constructive predecessorrestrial itadjusted, veterin MFTLostgmail quotas Muk
Ep 2 (Step 000010): Train loss 6.879, Val loss 7.214
Ep 2 (Step 000015): Train loss 6.332, Val loss 6.785
Every effort moves you the. that marble brav a to for probe have devil there be one he concedeYest Petraeus ourselves-- way past that curiosity still abouticated IJack  to rooting still had I sublimeroud an begins I want, his used creatureHas if--;
Ep 3 (Step 000020): Train loss 6.050, Val loss 6.763
Every effort moves you of a Baker the and always one more with up,fam: " the if that a down-- go had woman, heris him--ived to................................................................ curiosity own and over in one a

In [39]:
torch.save({
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict()
}, "model_And_optimizer_pth")

In [43]:
check_pt = torch.load("model_And_optimizer_pth")
model = GPT(cfg)
model.load_state_dict(check_pt["model_state_dict"])
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.1)
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

C:\Users\Kabir Ghuge\AppData\Local\Temp\ipykernel_20712\730699822.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  check_pt = torch.load("model_And_optimizer_pth")


RuntimeError: Error(s) in loading state_dict for GPT:
	Missing key(s) in state_dict: "tf_blocks.0.attention.Wq.bias", "tf_blocks.0.attention.Wk.bias", "tf_blocks.0.attention.Wv.bias", "tf_blocks.1.attention.Wq.bias", "tf_blocks.1.attention.Wk.bias", "tf_blocks.1.attention.Wv.bias", "tf_blocks.2.attention.Wq.bias", "tf_blocks.2.attention.Wk.bias", "tf_blocks.2.attention.Wv.bias", "tf_blocks.3.attention.Wq.bias", "tf_blocks.3.attention.Wk.bias", "tf_blocks.3.attention.Wv.bias", "tf_blocks.4.attention.Wq.bias", "tf_blocks.4.attention.Wk.bias", "tf_blocks.4.attention.Wv.bias", "tf_blocks.5.attention.Wq.bias", "tf_blocks.5.attention.Wk.bias", "tf_blocks.5.attention.Wv.bias", "tf_blocks.6.attention.Wq.bias", "tf_blocks.6.attention.Wk.bias", "tf_blocks.6.attention.Wv.bias", "tf_blocks.7.attention.Wq.bias", "tf_blocks.7.attention.Wk.bias", "tf_blocks.7.attention.Wv.bias", "tf_blocks.8.attention.Wq.bias", "tf_blocks.8.attention.Wk.bias", "tf_blocks.8.attention.Wv.bias", "tf_blocks.9.attention.W

In [31]:
pip install tensorflow >= 2.15.0 tqdm>=4.66


Note: you may need to restart the kernel to use updated packages.


In [32]:
import tensorflow as tf
import tqdm
print(tf.__version__)
print(tqdm.__version__)

2.19.0
4.67.1


In [33]:
import sys
sys.path.append("C:/Users/Kabir Ghuge/Downloads")
from gpt_download3 import download_and_load_gpt2

settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")
print(settings)
print(params.keys())

C:\Users\Kabir Ghuge\anaconda3\envs\torch-gpu\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


File already exists and is up-to-date: gpt2\124M\checkpoint


C:\Users\Kabir Ghuge\anaconda3\envs\torch-gpu\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


File already exists and is up-to-date: gpt2\124M\encoder.json


C:\Users\Kabir Ghuge\anaconda3\envs\torch-gpu\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

File already exists and is up-to-date: gpt2\124M\model.ckpt.index


C:\Users\Kabir Ghuge\anaconda3\envs\torch-gpu\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


File already exists and is up-to-date: gpt2\124M\model.ckpt.meta


C:\Users\Kabir Ghuge\anaconda3\envs\torch-gpu\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


File already exists and is up-to-date: gpt2\124M\vocab.bpe
{'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}
dict_keys(['blocks', 'b', 'g', 'wpe', 'wte'])


In [34]:
# update the cfg dictionary to match the gpt2 124M model
cfg.update({"context_length":1024, "qkv_bias": True})
gpt = GPT(cfg)
gpt.eval()

GPT(
  (embed_layer): Embedding(50257, 768)
  (pos_layer): Embedding(1024, 768)
  (dropout): Dropout(p=0.1, inplace=False)
  (tf_blocks): Sequential(
    (0): Transformer(
      (attention): MaskedMultiheadAttention(
        (Wq): Linear(in_features=768, out_features=768, bias=True)
        (Wk): Linear(in_features=768, out_features=768, bias=True)
        (Wv): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layer): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop): Dropout(p=0.1, inplace=False)
    )
    (1): Transformer(
      (attention): MaskedMultiheadAttention(
        (Wq): Linear(in_features=768, out_feature

In [35]:
# check wheter two tensors ro arrays(left and right) have same dim 
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch: left{left.shape}, right{right.shape}")
    return torch.nn.Parameter(torch.tensor(right))

In [36]:
def load_weights_into_gpt(gpt, params):
    gpt.pos_layer.weight = assign(gpt.pos_layer.weight, params['wpe'])
    gpt.embed_layer.weight = assign(gpt.embed_layer.weight, params['wte'])

    for b in range(len(params["blocks"])):
# weights of the multiheadattention block
        q_w, k_w, v_w = np.split(params["blocks"][b]["attn"]["c_attn"]["w"],3,axis=-1)
        gpt.tf_blocks[b].attention.Wq.weight = assign(gpt.tf_blocks[b].attention.Wq.weight, q_w.T)
        gpt.tf_blocks[b].attention.Wk.weight = assign(gpt.tf_blocks[b].attention.Wk.weight, k_w.T)
        gpt.tf_blocks[b].attention.Wv.weight = assign(gpt.tf_blocks[b].attention.Wv.weight, v_w.T)
# biases of the multiheadattention block
        q_b, k_b, v_b = np.split(params["blocks"][b]["attn"]["c_attn"]["b"],3,axis=-1)
        gpt.tf_blocks[b].attention.Wq.bias = assign(gpt.tf_blocks[b].attention.Wq.bias, q_b)
        gpt.tf_blocks[b].attention.Wk.bias = assign(gpt.tf_blocks[b].attention.Wk.bias, k_b)
        gpt.tf_blocks[b].attention.Wv.bias = assign(gpt.tf_blocks[b].attention.Wv.bias, v_b)
# weigthts and biases of the feedforward neural network
        gpt.tf_blocks[b].ff.layer[0].weight = assign(gpt.tf_blocks[b].ff.layer[0].weight, params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.tf_blocks[b].ff.layer[0].bias = assign(gpt.tf_blocks[b].ff.layer[0].bias, params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.tf_blocks[b].ff.layer[2].weight = assign(gpt.tf_blocks[b].ff.layer[2].weight, params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.tf_blocks[b].ff.layer[2].bias = assign(gpt.tf_blocks[b].ff.layer[2].bias, params["blocks"][b]["mlp"]["c_proj"]["b"])
# weight and biases of the linear layer of the transformer 
        gpt.tf_blocks[b].attention.out_proj.weight = assign(gpt.tf_blocks[b].attention.out_proj.weight, params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.tf_blocks[b].attention.out_proj.bias = assign(gpt.tf_blocks[b].attention.out_proj.bias, params["blocks"][b]["attn"]["c_proj"]["b"])
# linear normalization 1 and 2 (scale and shift parameters)
        gpt.tf_blocks[b].norm1.scale = assign(gpt.tf_blocks[b].norm1.scale, params["blocks"][b]["ln_1"]["g"])
        gpt.tf_blocks[b].norm1.shift = assign(gpt.tf_blocks[b].norm1.shift, params["blocks"][b]["ln_1"]["b"])
        gpt.tf_blocks[b].norm2.scale = assign(gpt.tf_blocks[b].norm2.scale, params["blocks"][b]["ln_2"]["g"])
        gpt.tf_blocks[b].norm2.shift = assign(gpt.tf_blocks[b].norm2.shift, params["blocks"][b]["ln_2"]["b"])
# final normalization scale and shift parameters
    gpt.norm.scale = assign(gpt.norm.scale, params["g"])
    gpt.norm.shift = assign(gpt.norm.shift, params["b"])
    gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"])

In [37]:
load_weights_into_gpt(gpt, params)
gpt.to(device);
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):

    # For-loop is the same as before: Get logits, and only focus on last time step
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]

        # New: Filter logits with top_k sampling
        if top_k is not None:
            # Keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float("-inf")).to(logits.device), logits)

        # New: Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # Apply softmax to get probabilities
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)

        # Otherwise same as before: get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

        if idx_next == eos_id:  # Stop generating early if end-of-sequence token is encountered and eos_id is specified
            break

        # Same as before: append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

    return idx

torch.manual_seed(123)

generated_tokens = generate(
    idx = text_to_token_ids("once in the bright day", tokenizer).to(device), model=gpt, max_new_tokens=100, context_size=cfg["context_length"], temperature=1, top_k=50
)

print("Output text:\n", token_ids_to_text(generated_tokens, tokenizer))

Output text:
 once in the bright day as we were waiting to get in the door. I looked up and saw a man's face. As I asked him I saw the name William Johnson from a distance and he answered: "My god you are going to be sorry that is not the last." Then he went and took out a couple of knives, which he carried back to our hotel which is on Broadway. "Why do you have them?" I asked. "Well what about those?" he said. "We'll make a deal
